In [123]:
import pandas as pd
import numpy as np
import pickle
import datetime
from functools import reduce
from scipy.stats import mode

# 데이터 불러오기

- 2015/01/01 ~ 2022/08/31 수도권 대기환경 관측 데이터

In [3]:
# 수도권 대기오염 데이터 불러오기
seoul_df = pd.read_csv("./data/preprodata/seoul_pollution_df.csv")
incheon_df = pd.read_csv("./data/preprodata/incheon_pollution_df.csv")
gyeongi_df = pd.read_csv("./data/preprodata/gyeongi_pollution_df.csv")

C:\Users\JONGYEOB\AppData\Local\Temp\ipykernel_9228\2035712611.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  seoul_df = pd.read_csv("./data/preprodata/seoul_pollution_df.csv")
C:\Users\JONGYEOB\AppData\Local\Temp\ipykernel_9228\2035712611.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  incheon_df = pd.read_csv("./data/preprodata/incheon_pollution_df.csv")
C:\Users\JONGYEOB\AppData\Local\Temp\ipykernel_9228\2035712611.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  gyeongi_df = pd.read_csv("./data/preprodata/gyeongi_pollution_df.csv")


# 1. datetime 컬럼 추가 및 불필요한 컬럼 제거
- 전처리 마지막 단계에서 datetime 기준으로 각 측정소별 관측치를 열 방향으로 병합할 예정
- 사용하지 않는 컬럼 제거

In [6]:
def add_datetime_remove_columns(df):
    
    # 데이터프레임의 측정일시를 리스트로 저장
    datetime_lst = df["측정일시"].tolist()
    
    # 형식을 맞춘 측정일시를 임시로 저장할 리스트 생성
    temp_datetime = []
    
    # 데이터프레임의 측정일시를 반복문으로 돌려 형식을 맞춰줌
    for datetime in datetime_lst:
        
        year = str(datetime)[:4]
        month = str(datetime)[4:6]
        day = str(datetime)[6:8]
        hour = str(int(str(datetime)[-2:]) - 1)
        
        # 생성했던 빈 리스트에 형식을 맞춘 date와 time 저장
        temp_datetime.append(f"{year}-{month}-{day} {hour}:00")
        
    # 데이터프레임에 datetime 컬럼 추가
    df["datetime"] = temp_datetime
    
    # datetime 컬럼을 datetime 형식으로 변경
    df["datetime"] = pd.to_datetime(df["datetime"])
    
    # 불필요한 컬럼 제거
    ndf = df.drop(["측정일시", "주소", "측정소코드", "DATA_TINE", "PM25", "망"], axis = 1)
    
    return ndf

In [54]:
seoul_ndf = add_datetime_remove_columns(seoul_df)
incheon_ndf = add_datetime_remove_columns(incheon_df)
gyeongi_ndf = add_datetime_remove_columns(gyeongi_df)

In [8]:
seoul_ndf.head()

,지역,측정소명,SO2,CO,O3,NO2,PM10,datetime
0,서울,중구,0.006,0.6,0.022,0.011,44.0,2015-01-01 00:00:00
1,서울,중구,0.006,0.6,0.022,0.010,57.0,2015-01-01 01:00:00
2,서울,중구,0.006,0.6,0.021,0.010,76.0,2015-01-01 02:00:00
3,서울,중구,0.006,0.6,0.022,0.009,86.0,2015-01-01 03:00:00
4,서울,중구,0.006,0.5,0.021,0.009,106.0,2015-01-01 04:00:00


# 2. 측정소명 기준으로 데이터 나누기
- 측정소명을 기준으로 컬럼명을 지정하고, 관측값의 결측치 처리를 위해 데이터 분할
- 2015년 이후에 생긴 측정소의 경우엔, 데이터의 행 수가 다른 측정소와 맞지 않기 때문에 제외시킴

In [14]:
def divide_by_station(df, city = "seoul"):
    
    # 측정소명 리스트 생성
    total_stations = df["측정소명"].unique().tolist()
    
    # 2015년 이후에 생긴 측정소
    del_lst = ["시흥대로", "부평역", "서해", "중봉", "길상", "삼산", "서창", "아암", "영종", "영흥", "주안", "효성", "경인항", 
               "인천 남항", "인천 북항", "인천 신항", "인천항", "연평도", "울도", "가남읍","고덕면","고읍","곤지암","공도읍",
               "대신면","모현읍","목감동","미사","배곧동","백암면","별내동", "봉담읍","부발읍","새솔동","서신면","설악면",
               "송북동",'송산3동','신원동',"양평읍","오포1동","와부읍", "우정읍","이동읍","일동면","장현동","장호원읍","전곡",
               "주엽동","죽산면","진접읍","청계동","청북읍","파주읍", "한강신도시","호매실","화도읍","연천(DMZ)","파주","경춘로",
               "금암로(신장동)","서해안로","한강로"]
    
    # 측정소명을 반복문으로 돌려 데이터 분할
    for station in total_stations:
        
        if station not in del_lst:
        
            # 각 측정소의 인덱스 번호 가져오기 (데이터프레임 이름 변경을 위해)
            idx = total_stations.index(station)

            # 측정소명 별로 데이터 분할
            globals()[f"df_{city}_{idx}"] = df[df["측정소명"] == station]

In [77]:
divide_by_station(seoul_ndf, "seoul")
divide_by_station(incheon_ndf, "incheon")
divide_by_station(gyeongi_ndf, "gyeongi")

In [19]:
df_seoul_0.head()

,지역,측정소명,SO2,CO,O3,NO2,PM10,datetime
0,서울,중구,0.006,0.6,0.022,0.011,44.0,2015-01-01 00:00:00
1,서울,중구,0.006,0.6,0.022,0.010,57.0,2015-01-01 01:00:00
2,서울,중구,0.006,0.6,0.021,0.010,76.0,2015-01-01 02:00:00
3,서울,중구,0.006,0.6,0.022,0.009,86.0,2015-01-01 03:00:00
4,서울,중구,0.006,0.5,0.021,0.009,106.0,2015-01-01 04:00:00


In [20]:
df_incheon_0.head()

,지역,측정소명,SO2,CO,O3,NO2,PM10,datetime
0,인천 강화군,송해,0.007,0.2,0.015,0.004,73.0,2015-01-01 00:00:00
1,인천 강화군,송해,0.007,0.2,0.015,0.005,93.0,2015-01-01 01:00:00
2,인천 강화군,송해,0.007,0.1,0.015,0.004,100.0,2015-01-01 02:00:00
3,인천 강화군,송해,0.006,0.1,0.014,0.004,111.0,2015-01-01 03:00:00
4,인천 강화군,송해,0.007,0.1,0.014,0.004,131.0,2015-01-01 04:00:00


In [21]:
df_gyeongi_0.head()

,지역,측정소명,SO2,CO,O3,NO2,PM10,datetime
0,경기 가평군,가평,0.006,0.3,0.024,0.005,67.0,2015-01-01 00:00:00
1,경기 가평군,가평,0.006,0.3,0.023,0.005,78.0,2015-01-01 01:00:00
2,경기 가평군,가평,0.005,0.3,0.023,0.005,79.0,2015-01-01 02:00:00
3,경기 가평군,가평,0.006,0.3,0.022,0.005,101.0,2015-01-01 03:00:00
4,경기 가평군,가평,0.006,0.3,0.022,0.005,102.0,2015-01-01 04:00:00


# 3. 결측치 처리
- SO2, CO, O3, NO2, PM10은 1시간 단위의 관측값이므로 이전 값과 이후 값의 평균으로 결측치를 대체
- 측정소 별로 데이터를 먼저 분할한 이유는 결측치 대체 시 다른 측정소 관측값에 영향을 받지 않게 하기 위함

In [76]:
def interpolate_missing_values(city = "seoul"):
    
    # 결측치 처리할 컬럼명
    interpolate_columns = ["SO2", "CO", "O3", "NO2", "PM10"]
    
    # 2015년 이후에 생긴 측정소
    del_lst = ["시흥대로", "부평역", "서해", "중봉", "길상", "삼산", "서창", "아암", "영종", "영흥", "주안", "효성", "경인항", 
               "인천 남항", "인천 북항", "인천 신항", "인천항", "연평도", "울도", "가남읍","고덕면","고읍","곤지암","공도읍",
               "대신면","모현읍","목감동","미사","배곧동","백암면","별내동", "봉담읍","부발읍","새솔동","서신면","설악면",
               "송북동",'송산3동','신원동',"양평읍","오포1동","와부읍", "우정읍","이동읍","일동면","장현동","장호원읍","전곡",
               "주엽동","죽산면","진접읍","청계동","청북읍","파주읍", "한강신도시","호매실","화도읍","연천(DMZ)","파주","경춘로",
               "금암로(신장동)","서해안로","한강로"]
    
    # 지역별 측정소명 리스트 생성
    seoul_stations = seoul_ndf["측정소명"].unique().tolist()
    incheon_stations = incheon_ndf["측정소명"].unique().tolist()
    gyeongi_stations = gyeongi_ndf["측정소명"].unique().tolist()
    
    # 각 지역별 측정소명 리스트 가져오기
    if city == "seoul":
        total_stations = seoul_stations
    elif city == "incheon":
        total_stations = incheon_stations
    else :
        total_stations = gyeongi_stations
        
    for station in total_stations:
        
        if station not in del_lst:
        
            # 각 측정소의 인덱스 번호 가져오기
            idx = total_stations.index(station)

            # 각 측정소 데이터 가져오기
            temp_df = globals()[f"df_{city}_{idx}"].copy()

            for column in interpolate_columns:
                temp_df[column].interpolate(inplace = True)

            # 인덱스 번호로 컬럼명 변경 (이후에 열 방향으로 병합하기 위해)
            temp_df = temp_df.rename(columns = {
                "SO2" : f"SO2_{city}_{idx}",
                "CO" : f"CO_{city}_{idx}",
                "O3" : f"O3_{city}_{idx}",
                "NO2" : f"NO2_{city}_{idx}",
                "PM10" : f"PM10_{city}_{idx}"
            })

            globals()[f"df_{city}_{idx}"] = temp_df

In [78]:
interpolate_missing_values("seoul")
interpolate_missing_values("incheon")
interpolate_missing_values("gyeongi")

In [79]:
df_seoul_0.head()

,지역,측정소명,SO2_seoul_0,CO_seoul_0,O3_seoul_0,NO2_seoul_0,PM10_seoul_0,datetime
0,서울,중구,0.006,0.6,0.022,0.011,44.0,2015-01-01 00:00:00
1,서울,중구,0.006,0.6,0.022,0.010,57.0,2015-01-01 01:00:00
2,서울,중구,0.006,0.6,0.021,0.010,76.0,2015-01-01 02:00:00
3,서울,중구,0.006,0.6,0.022,0.009,86.0,2015-01-01 03:00:00
4,서울,중구,0.006,0.5,0.021,0.009,106.0,2015-01-01 04:00:00


In [80]:
df_incheon_0.head()

,지역,측정소명,SO2_incheon_0,CO_incheon_0,O3_incheon_0,NO2_incheon_0,PM10_incheon_0,datetime
0,인천 강화군,송해,0.007,0.2,0.015,0.004,73.0,2015-01-01 00:00:00
1,인천 강화군,송해,0.007,0.2,0.015,0.005,93.0,2015-01-01 01:00:00
2,인천 강화군,송해,0.007,0.1,0.015,0.004,100.0,2015-01-01 02:00:00
3,인천 강화군,송해,0.006,0.1,0.014,0.004,111.0,2015-01-01 03:00:00
4,인천 강화군,송해,0.007,0.1,0.014,0.004,131.0,2015-01-01 04:00:00


In [81]:
df_gyeongi_0.head()

,지역,측정소명,SO2_gyeongi_0,CO_gyeongi_0,O3_gyeongi_0,NO2_gyeongi_0,PM10_gyeongi_0,datetime
0,경기 가평군,가평,0.006,0.3,0.024,0.005,67.0,2015-01-01 00:00:00
1,경기 가평군,가평,0.006,0.3,0.023,0.005,78.0,2015-01-01 01:00:00
2,경기 가평군,가평,0.005,0.3,0.023,0.005,79.0,2015-01-01 02:00:00
3,경기 가평군,가평,0.006,0.3,0.022,0.005,101.0,2015-01-01 03:00:00
4,경기 가평군,가평,0.006,0.3,0.022,0.005,102.0,2015-01-01 04:00:00


# 4. 결측치 확인 및 재처리

In [82]:
# 2015년 측정소를 제외한 측정소 리스트 생성
seoul_stations = seoul_ndf["측정소명"].unique().tolist()
incheon_stations = incheon_ndf["측정소명"].unique().tolist()
gyeongi_stations = gyeongi_ndf["측정소명"].unique().tolist()

del_lst = ["시흥대로", "부평역", "서해", "중봉", "길상", "삼산", "서창", "아암", "영종", "영흥", "주안", "효성", "경인항", 
           "인천 남항", "인천 북항", "인천 신항", "인천항", "연평도", "울도", "가남읍","고덕면","고읍","곤지암","공도읍",
           "대신면","모현읍","목감동","미사","배곧동","백암면","별내동", "봉담읍","부발읍","새솔동","서신면","설악면",
           "송북동",'송산3동','신원동',"양평읍","오포1동","와부읍", "우정읍","이동읍","일동면","장현동","장호원읍","전곡",
           "주엽동","죽산면","진접읍","청계동","청북읍","파주읍", "한강신도시","호매실","화도읍","연천(DMZ)","파주","경춘로",
           "금암로(신장동)","서해안로","한강로"]

seoul_lst = [x for x in seoul_stations if x not in del_lst]
incheon_lst = [x for x in incheon_stations if x not in del_lst]
gyeongi_lst = [x for x in gyeongi_stations if x not in del_lst]

In [83]:
# 경기 결측치 확인
for station in gyeongi_lst:
    
    # 각 측정소의 인덱스 번호 가져오기
    idx = gyeongi_lst.index(station)
    
    print("=" * 20)
    print(idx)
    print(station)
    print("지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime")
    print(globals()["df_gyeongi_{}".format(idx)].isna().sum().tolist())
    print(len(globals()["df_gyeongi_{}".format(idx)]))
    print("=" * 20)

0
가평
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
58416
1
행신동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 108, 0, 0, 0]
67200
2
식사동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
67200
3
마두역
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
17544
4
별양동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
67200
5
과천동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
67200
6
철산동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
67200
7
소하동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
67200
8
경안동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
67200
9
교문동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 110, 0, 0, 0, 0]
67200
10
동구동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // da

[0, 0, 0, 0, 0, 0, 0, 0]
67200
78
남양읍
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
58416
79
향남
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
19704
80
동탄
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
58416
81
가평읍
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
82
고촌면
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
83
정자1동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
84
상대원1동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
85
시화공단
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
86
남양동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
87
동탄동
지역 // 측정소명 // SO2 // CO // O3 // NO2 // PM10 // datetime
[0, 0, 0, 0, 0, 0, 0, 0]
8784
88
경수대로(동수원)
지역 // 측정소명 

- 행 길이가 맞지 않는 다수의 데이터프레임 발견
    - 67200행보다 많은 경우는 데이터를 분할하면서 문제가 생긴 것으로 판단
    - 또한, 경기도의 다수의 측정소에서 공통적으로 2016년 데이터가 누락된 것을 확인
        - 확인해본 결과, 2016년만 측정소명이 조금씩 다르게 표현된 것을 확인함

            - 가평 - 가평읍
            - 고촌읍 - 고촌면
            - 정자동 - 정자1동
            - 상대원동 - 상대원1동
            - 시화산단 - 시화공단
            - 남양읍 - 남양동
            - 동탄 - 동탄동

## 4-1. 행이 기준보다 많은 데이터 처리
- 2015/01/01 ~ 2022/08/31까지 데이터가 전부 있다면 67200행이어야 함

In [84]:
len(df_incheon_1), len(df_incheon_17)

(92784, 92784)

In [85]:
df_incheon_1.drop_duplicates(subset=['datetime'], keep='first', ignore_index=True, inplace = True)
df_incheon_17.drop_duplicates(subset=['datetime'], keep='first', ignore_index=True, inplace = True)

In [86]:
len(df_incheon_1), len(df_incheon_17)

(67200, 67200)

## 4-2. 경기도 측정소명이 다르게 표현된 데이터 병합

In [87]:
changed_stations = {"가평" : "가평읍", 
                    "고촌읍" : "고촌면", 
                    "정자동" : "정자1동",
                    "상대원동" : "상대원1동",
                    "시화산단" : "시화공단", 
                    "남양읍" : "남양동",
                    "동탄" : "동탄동"}

for station in changed_stations.keys():
    
    # 채워야 할 데이터프레임의 인덱스 번호
    idx = gyeongi_stations.index(station)
    
    # 2016년에 이름이 바뀐 데이터프레임 인덱스 번호
    changed_station = changed_stations[station]
    changed_idx = gyeongi_stations.index(changed_station)
    
    # 채워야 할 데이터프레임과 이름이 바뀐 데이터프레임 가져오기
    temp_df = globals()["df_gyeongi_{}".format(idx)].copy()
    changed_df = globals()["df_gyeongi_{}".format(changed_idx)].copy()
    
    # 인덱스 번호로 컬럼명 변경
    changed_df = changed_df.rename(columns = {
        "SO2_gyeongi_{}".format(changed_idx) : "SO2_gyeongi_{}".format(idx),
        "CO_gyeongi_{}".format(changed_idx) : "CO_gyeongi_{}".format(idx),
        "O3_gyeongi_{}".format(changed_idx) : "O3_gyeongi_{}".format(idx),
        "NO2_gyeongi_{}".format(changed_idx) : "NO2_gyeongi_{}".format(idx),
        "PM10_gyeongi_{}".format(changed_idx) : "PM10_gyeongi_{}".format(idx)
    })
    
    # 데이터 병합
    globals()["df_gyeongi_{}".format(idx)] = pd.concat([temp_df, changed_df], axis = 0)

In [88]:
len(df_gyeongi_0)

67200

# 5. 전체 행 수가 67200개 미만인 데이터 제외하고 병합
- 2015년 1월 1일부터 2022년 08월 31일까지 데이터가 존재할 경우, 67200행이 나와야 함
- 67200행이 아닐 경우, 2015년 이후 생겼거나 중간에 사라진 측정소라고 판단
- datetime을 기준으로 열 방향으로 병합

In [109]:
def merge_data(city = "seoul"):

    # 데이터를 병합해서 담을 all_df를 미리 생성하기 (datetime을 기준으로 병합할 예정이므로 datetime을 미리 넣어놓기)
    all_df = pd.DataFrame(df_seoul_0["datetime"])
    
    # 지역별 측정소명 리스트 생성
    seoul_stations = seoul_ndf["측정소명"].unique().tolist()
    incheon_stations = incheon_ndf["측정소명"].unique().tolist()
    gyeongi_stations = gyeongi_ndf["측정소명"].unique().tolist()
    
    # 사용하지 않는 측정소 리스트
    del_lst = ["시흥대로", "부평역", "서해", "중봉", "길상", "삼산", "서창", "아암", "영종", "영흥", "주안", "효성", "경인항", 
               "인천 남항", "인천 북항", "인천 신항", "인천항", "연평도", "울도", "가남읍","고덕면","고읍","곤지암","공도읍",
               "대신면","모현읍","목감동","미사","배곧동","백암면","별내동", "봉담읍","부발읍","새솔동","서신면","설악면",
               "송북동",'송산3동','신원동',"양평읍","오포1동","와부읍", "우정읍","이동읍","일동면","장현동","장호원읍","전곡",
               "주엽동","죽산면","진접읍","청계동","청북읍","파주읍", "한강신도시","호매실","화도읍","연천(DMZ)","파주","경춘로",
               "금암로(신장동)","서해안로","한강로"]
    
    # 각 지역별 측정소명 리스트 가져오기
    if city == "seoul":
        total_stations = seoul_stations
    elif city == "incheon":
        total_stations = incheon_stations
    else :
        total_stations = gyeongi_stations
        
    for station in total_stations:
        
        if station not in del_lst:
        
            # 각 측정소의 인덱스 번호 가져오기
            idx = total_stations.index(station)

            # 각 측정소 데이터 가져오기 (이제 필요없는 지역과 측정소명 제외)
            temp_df = globals()[f"df_{city}_{idx}"].drop(["지역", "측정소명"], axis = 1)
            
            # 가져온 데이터의 행이 67200개일 경우, datetime을 기준으로 병합
            if len(temp_df) == 67200:
                all_df = pd.merge(all_df, temp_df, how = 'outer', on = 'datetime')
                
    return all_df

In [110]:
all_df_seoul = merge_data("seoul")
all_df_incheon = merge_data("incheon")
all_df_gyeongi = merge_data("gyeongi")

In [124]:
all_df = reduce(lambda left, right : pd.merge(left, right, on = "datetime"), [all_df_seoul, all_df_incheon, all_df_gyeongi])

In [125]:
all_df.shape

(67200, 636)

# 5. 병합한 데이터 결측치 재확인

In [126]:
non_data_lst = all_df.isna().sum().tolist()

for idx in range(len(non_data_lst)):
    
    if non_data_lst[idx] > 0:
        print(all_df.columns[idx])
        print(non_data_lst[idx])
        print("=" * 25)

CO_seoul_19
14
O3_gyeongi_1
108
CO_gyeongi_9
110
CO_gyeongi_21
48
PM10_gyeongi_39
2
CO_gyeongi_45
56784
CO_gyeongi_54
24
PM10_gyeongi_72
120
SO2_gyeongi_73
35


- 결측치가 56784개로 상당히 많은 gyeongi_45 데이터 삭제
- 나머지는 이후 값 대체 (첫 행이 결측치인 경우, 이전 값이 존재하지 않아서 interpolate로 채워지지 않는 문제)
    - 다른 측정소 데이터에 영향을 받을 수 있지만 비율이 0.2%도 안 되기 때문에 큰 영향을 끼치지 않을 것이라고 판단

## 5-2. 결측치가 많은 측정소 데이터 삭제

In [127]:
df_gyeongi_45.head()

,지역,측정소명,SO2_gyeongi_45,CO_gyeongi_45,O3_gyeongi_45,NO2_gyeongi_45,PM10_gyeongi_45,datetime
97200,경기 안산시,원곡동,0.004,NaN,0.024,0.008,58.0,2015-01-01 00:00:00
97201,경기 안산시,원곡동,0.004,NaN,0.022,0.009,85.0,2015-01-01 01:00:00
97202,경기 안산시,원곡동,0.004,NaN,0.021,0.009,97.0,2015-01-01 02:00:00
97203,경기 안산시,원곡동,0.004,NaN,0.022,0.008,112.0,2015-01-01 03:00:00
97204,경기 안산시,원곡동,0.004,NaN,0.020,0.009,116.0,2015-01-01 04:00:00


In [128]:
drop_cols = ["SO2_gyeongi_45", "CO_gyeongi_45", "O3_gyeongi_45", "NO2_gyeongi_45", "PM10_gyeongi_45"]
all_df = all_df.drop(drop_cols, axis = 1)

## 5-3. 결측치 이후 값으로 대체

In [129]:
all_df.fillna(method='bfill', inplace = True)

In [130]:
non_data_lst = all_df.isna().sum().tolist()
cnt = 0

for idx in range(len(non_data_lst)):
    
    if non_data_lst[idx] > 0:
        print(all_df.columns[idx])
        print(non_data_lst[idx])
        print("=" * 25)
        
    elif non_data_lst[idx] == 0:
        cnt += 1
        
print(cnt)

631


In [131]:
len(all_df.columns)

631

- 결측치 전부 해결됨

# 6. 종속변수 추가

- 서울시 종로구의 익일 PM10을 종속변수로 추가

## 6-1. 익일 종로구 PM10을 종속변수로 추가

In [132]:
seoul_stations.index("종로구")

2

In [134]:
# all_df의 datetime과 종로구 PM10을 추출해서 저장
PM10_y = all_df[["datetime", "PM10_seoul_2"]]

# 추출한 PM10_y의 datetime을 하루 전으로 설정해서 변수에 저장
yesterday = PM10_y["datetime"] - pd.Timedelta(1, unit = "days")

# PM10_y의 datetime을 하루 전으로 설정한 값으로 변경
PM10_y["datetime"] = yesterday

# 컬럼명 변경
PM10_y = PM10_y.rename(columns = {"PM10_seoul_2" : "PM10_y"})

# datetime을 기준으로 all_df에 병합
final_df = pd.merge(all_df, PM10_y, on = "datetime", how = "inner")

C:\Users\JONGYEOB\AppData\Local\Temp\ipykernel_9228\3196284066.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10_y["datetime"] = yesterday


In [135]:
final_df

,datetime,SO2_seoul_0,CO_seoul_0,O3_seoul_0,NO2_seoul_0,PM10_seoul_0,SO2_seoul_1,CO_seoul_1,O3_seoul_1,NO2_seoul_1,...,CO_gyeongi_78,O3_gyeongi_78,NO2_gyeongi_78,PM10_gyeongi_78,SO2_gyeongi_80,CO_gyeongi_80,O3_gyeongi_80,NO2_gyeongi_80,PM10_gyeongi_80,PM10_y
0,2015-01-01 00:00:00,0.006,0.6,0.022,0.011,44.0,0.005,0.5,0.011,0.028,...,0.3,0.023,0.006,41.0,0.007,0.1,0.034,0.008,47.0,26.0
1,2015-01-01 01:00:00,0.006,0.6,0.022,0.010,57.0,0.005,0.4,0.013,0.021,...,0.3,0.021,0.006,75.0,0.007,0.1,0.034,0.007,53.0,23.0
2,2015-01-01 02:00:00,0.006,0.6,0.021,0.010,76.0,0.004,0.3,0.015,0.016,...,0.3,0.022,0.005,105.0,0.007,0.1,0.032,0.007,75.0,23.0
3,2015-01-01 03:00:00,0.006,0.6,0.022,0.009,86.0,0.004,0.4,0.016,0.014,...,0.3,0.021,0.005,117.0,0.007,0.1,0.032,0.006,86.0,24.0
4,2015-01-01 04:00:00,0.006,0.5,0.021,0.009,106.0,0.004,0.3,0.017,0.010,...,0.2,0.020,0.005,122.0,0.007,0.1,0.031,0.006,94.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67171,2022-08-30 19:00:00,0.002,0.4,0.011,0.026,8.0,0.004,0.5,0.006,0.028,...,0.3,0.006,0.026,7.0,0.002,0.2,0.009,0.026,4.0,7.0
67172,2022-08-30 20:00:00,0.002,0.4,0.011,0.023,8.0,0.003,0.5,0.005,0.026,...,0.3,0.008,0.021,6.0,0.002,0.2,0.014,0.017,5.0,8.0
67173,2022-08-30 21:00:00,0.002,0.4,0.012,0.021,5.0,0.003,0.4,0.007,0.024,...,0.3,0.008,0.020,6.0,0.002,0.2,0.014,0.017,4.0,7.0
67174,2022-08-30 22:00:00,0.002,0.3,0.015,0.017,5.0,0.004,0.4,0.007,0.022,...,0.2,0.009,0.018,6.0,0.002,0.2,0.015,0.014,8.0,10.0


## 6-2. 종속변수 범주화
- 기상청에서 제시하는 기준에 따라 종속변수를 범주화 (0: 좋음, 1: 보통, 2: 나쁨, 3: 매우 나쁨)

In [136]:
bins = [0, 30, 80, 150, 1160]
cut_df_pm10 = pd.cut(final_df["PM10_y"], bins = bins, labels = [0, 1, 2, 3])
final_df["PM10_y_bin"] = cut_df_pm10
final_df.drop("PM10_y", axis = 1, inplace = True)

In [137]:
np.unique(final_df["PM10_y_bin"])

array([0, 1, 2, 3], dtype=int64)

# 7. 파일 저장

In [138]:
final_df.to_csv("./data/preprodata/preprocessed_pollutant_df_ver1.csv", index = False)